In [ ]:
# 라이브러리 불러오기
import folium
# 서울 지도 만들기
seoul_map = folium.Map(location=[37.5802219,126.9226047], zoom_start=18)
# 지도를 HTML 파일로 저장하기 #seoul_map.save('./seoul.html’)
seoul_map

In [ ]:
import pandas as pd
import folium

df = pd.read_excel('/content/seoulUniv.xlsx', engine= 'openpyxl')
# 서울 지도 만들기
seoul_map = folium.Map(location=[37.55,126.98], zoom_start=12)
# 대학교 위치정보를 Marker로 표시
for name, lat, lng in zip(df.index, df.위도, df.경도):
  folium.Marker([lat, lng], popup=name).add_to(seoul_map)
seoul_map

In [ ]:
df = pd.read_excel('/content/seoulUniv.xlsx', engine= 'openpyxl')

# 서울 지도 만들기
seoul_map = folium.Map(location=[37.55,126.98], zoom_start=12)
seoul_map

for name, lat, lng in zip(df.index, df.위도, df.경도):
  folium.CircleMarker([lat, lng],
    radius=10, # 원의 반지름
    color='brown', # 원의 둘레 색상
    fill=True,
    fill_color='coral', # 원을 채우는 색
    fill_opacity=0.7, # 투명도
    popup=name
  ).add_to(seoul_map)

# 지도를 HTML 파일로 저장하기.
seoul_map

# 스타벅스 크롤링 -> 지오 데이터 -> 지도에 마커찍기

In [ ]:
# request = requests.post(targetSite, data={
#     'ins_lat'   : 37.5108295,           # 위도
#     'ins_lng'   : 127.02928809999999,   # 경도
#     'p_sido_cd' : '01',                 # 시,도 분류 코드
#     'p_gugun_cd': '',                   # 시,군,구 분류 코드
#     'in_biz_cd' : '' ,                  # 알 수 없는 정보(불필요)
#     'iend'      : 2000,                 # 불러올 최대 매장 수
#     'set_date'  : ''                    # 알 수 없는 정보(불필요)
# })

In [ ]:
import warnings
import requests
import folium
from pandas.io.json import json_normalize

# 스타벅스 사이트에서 위치정보를 수집
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=5XLKAT2F0K'
request = requests.post(targetSite, data={
    'ins_lat'   : 37.5108295,           # 위도
    'ins_lng'   : 127.02928809999999,   # 경도
    'p_sido_cd' : '01',                 # 시,도 분류 코드
    'p_gugun_cd': '',                   # 시,군,구 분류 코드
    'in_biz_cd' : '' ,                  # 알 수 없는 정보(불필요)
    'iend'      : 2000,                 # 불러올 최대 매장 수
    'set_date'  : ''                    # 알 수 없는 정보(불필요)
})

star_json = request.json()
star_df = json_normalize(star_json, 'list')


<ipython-input-64-e64239d48825>:19: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  star_df = json_normalize(star_json, 'list')


In [ ]:
star_df = json_normalize(star_json, 'list')
print(type(star_df))
print(len(star_df))
star_df # 타입이 맞지 않아 None

<ipython-input-66-daccd2f8c8c0>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  star_df = json_normalize(star_json, 'list')


<class 'pandas.core.frame.DataFrame'>
612


,seq,sido_cd,sido_nm,gugun_cd,gugun_nm,code_order,view_yn,store_num,sido,gugun,...,p90,p01,t05,t30,t36,t27,t29,t43,t48,z9999
0,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
1,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
2,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
3,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
4,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
608,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
609,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
610,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
star_df_map = star_df[['s_name', 'sido_code', 'sido_name', 'gugun_code', 'gugun_name'
                       , 'doro_address', 'lat', 'lot']]

# astype() 메소드로 위도, 경도의 데이터 타입 —> object에서 float.
star_df_map['lat'] = star_df_map['lat'].astype(float)
star_df_map['lot'] = star_df_map['lot'].astype(float)

<ipython-input-67-10028af16f16>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  star_df_map['lat'] = star_df_map['lat'].astype(float)
<ipython-input-67-10028af16f16>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  star_df_map['lot'] = star_df_map['lot'].astype(float)


In [ ]:
star_df_map

,s_name,sido_code,sido_name,gugun_code,gugun_name,doro_address,lat,lot
0,역삼아레나빌딩,01,서울,0101,강남구,서울특별시 강남구 언주로 425 (역삼동),37.501087,127.043069
1,논현역사거리,01,서울,0101,강남구,서울특별시 강남구 강남대로 538 (논현동),37.510178,127.022223
2,신사역성일빌딩,01,서울,0101,강남구,서울특별시 강남구 강남대로 584 (논현동),37.513931,127.020606
3,국기원사거리,01,서울,0101,강남구,서울특별시 강남구 테헤란로 125 (역삼동),37.499517,127.031495
4,대치재경빌딩,01,서울,0101,강남구,서울특별시 강남구 남부순환로 2947 (대치동),37.494668,127.062583
...,...,...,...,...,...,...,...,...
607,사가정역,01,서울,0125,중랑구,서울특별시 중랑구 면목로 310,37.579594,127.087966
608,상봉역,01,서울,0125,중랑구,서울특별시 중랑구 망우로 307 (상봉동),37.596890,127.086470
609,묵동,01,서울,0125,중랑구,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",37.615368,127.076633
610,양원역,01,서울,0125,중랑구,서울특별시 중랑구 양원역로10길 3 (망우동),37.606654,127.106360


In [ ]:
star_map = folium.Map(
    location=[star_df_map['lat'].mean(),star_df_map['lot'].mean()],
    zoom_start=12)
for (index, data) in star_df_map.iterrows():
    string = '{}점 - {}'.format(data['s_name'], data['doro_address'])
    popup = folium.Popup(string, max_width=600)
    folium.Marker(
        location=[data['lat'], data['lot']],
        popup=popup
    ).add_to(star_map)
star_map

In [ ]:
# 데이터 받아와서
# matplotlib, seaborn, 지도(folium)위에 시각화 해보기